In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [10]:
import numpy as np 
import matplotlib.pyplot as plt


if torch.cuda.is_available():
    device = 'cuda'
if torch.backends.mps.is_available():
    device = 'mps'

In [24]:
class FVSBN(nn.Module):
    def __init__(self, n_dim = 784): 
        super(FVSBN, self).__init__()
        self.n_dim = n_dim
        self.linear_layers = nn.ModuleList(
            nn.Linear(in_features = max(1,i), out_features = 1) for i in range(n_dim)
        )

    
    def forward(self, input): 
        original_shape = input.shape
        input = input.view(original_shape[0], -1)

        output = [self.linear_layers[0](torch.zeros(original_shape[0], 1, device = device))]

        for i in range(1, self.n_dim): 
            output.append(self.linear_layers[i](input[:,:i]))
        output = torch.stack(output, axis = 1)
        return output
    

    def sample(self, x, output_dim):
        """
        Sequentially go through every single sample
        
        """
        sampled_image = torch.zeros(output_dim).to(device)
        sampled_image[0] = x
        
        for pixel in range(1,784):
            
            new_pixel = sampled_image.matmul(self.weights[:,pixel]).to(device)
            sampled_image[pixel] = new_pixel
            
        return sampled_image




In [20]:
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root = './data', train = True, download = True , transform = transform)
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)



device = torch.device(device)
print(device)


model = FVSBN(784)
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

num_epochs = 50

for epoch in range(num_epochs):
    total_loss = 0.0
    for images, _ in train_loader:
        optimizer.zero_grad()
        images = images.view(images.shape[0], -1)
        images = images.to(device)
        output = torch.squeeze(model(images))
        output.to(device)
        loss = criterion(output, images)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Calculate the average loss for the epoch
    epoch_loss = total_loss/ images.shape[0]
    

    # Print the loss for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

mps
Epoch [1/50], Loss: 4.4291
Epoch [2/50], Loss: 2.9697
Epoch [3/50], Loss: 2.8203
Epoch [4/50], Loss: 2.7739
Epoch [5/50], Loss: 2.7549
Epoch [6/50], Loss: 2.7453
Epoch [7/50], Loss: 2.7399
Epoch [8/50], Loss: 2.7372
Epoch [9/50], Loss: 2.7350
Epoch [10/50], Loss: 2.7335
Epoch [11/50], Loss: 2.7325
Epoch [12/50], Loss: 2.7319


KeyboardInterrupt: 

In [23]:
new_image = model.sample(0.3,784)
image_2d = new_image.view(28,28)
img = image_2d.cpu().detach().numpy()
plt.figure(figsize=(1, 1))
plt.imshow(img, cmap = 'gray')
plt.axis('off')  # Turn off axis
plt.show()

AttributeError: 'FVSBN' object has no attribute 'sample'